# Projet web scrapping sur le site de yuka sur 8000 poduits 

In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import time

In [2]:
url=[]#inistialiser une liste 
for i in range(1,7972):
    url.append('https://fr.openfoodfacts.org/'+str(i)) # stocker la forme de l'url de chaque page 

In [3]:
url[7900:7971]

['https://fr.openfoodfacts.org/7901',
 'https://fr.openfoodfacts.org/7902',
 'https://fr.openfoodfacts.org/7903',
 'https://fr.openfoodfacts.org/7904',
 'https://fr.openfoodfacts.org/7905',
 'https://fr.openfoodfacts.org/7906',
 'https://fr.openfoodfacts.org/7907',
 'https://fr.openfoodfacts.org/7908',
 'https://fr.openfoodfacts.org/7909',
 'https://fr.openfoodfacts.org/7910',
 'https://fr.openfoodfacts.org/7911',
 'https://fr.openfoodfacts.org/7912',
 'https://fr.openfoodfacts.org/7913',
 'https://fr.openfoodfacts.org/7914',
 'https://fr.openfoodfacts.org/7915',
 'https://fr.openfoodfacts.org/7916',
 'https://fr.openfoodfacts.org/7917',
 'https://fr.openfoodfacts.org/7918',
 'https://fr.openfoodfacts.org/7919',
 'https://fr.openfoodfacts.org/7920',
 'https://fr.openfoodfacts.org/7921',
 'https://fr.openfoodfacts.org/7922',
 'https://fr.openfoodfacts.org/7923',
 'https://fr.openfoodfacts.org/7924',
 'https://fr.openfoodfacts.org/7925',
 'https://fr.openfoodfacts.org/7926',
 'https://fr

## 1- definition d'une fonction qui recupere tout les urls des produits 

In [30]:
def recup_url(nb_page):
    liste_url_S=[]             #initialisation d'une liste 
    for i in range(1,nb_page+1):
        url_page='https://fr.openfoodfacts.org/'+str(i) # forme d 'url d'une page
        page=BeautifulSoup(requests.get(url_page).text,'html.parser')# faire comprendre a python que c est un texte html
        for a in page('a',href=True):# trouver toute les url de la page 
                if '/produit/' in a['href']:# limiter la recherche pour trouver juste les url ou on trouve produit 
                    liste_url_S.append('https://fr.openfoodfacts.org'+a['href']) # stocker dans la liste 
    return liste_url_S # retour de la fonction est notre liste 

In [31]:
recup_url(5)

['https://fr.openfoodfacts.org/produit/3274080005003/cristaline-eau-de-source',
 'https://fr.openfoodfacts.org/produit/3017620422003/nutella-pate-a-tartiner-noisettes-cacao-t-400-pot-de-400-gr-ferrero',
 'https://fr.openfoodfacts.org/produit/7622210449283/prince-chocolat-lu',
 'https://fr.openfoodfacts.org/produit/5449000000996/coca-cola',
 'https://fr.openfoodfacts.org/produit/3017620425035/nutella-pate-a-tartiner-noisettes-cacao-t-1000-pot-de-1-kg-ferrero',
 'https://fr.openfoodfacts.org/produit/3229820129488/muesli-raisin-figue-abricot-bjorg',
 'https://fr.openfoodfacts.org/produit/3175680011480/biscuit-sesame-gerble',
 'https://fr.openfoodfacts.org/produit/8000500310427/nutella-biscuits-ferrero',
 'https://fr.openfoodfacts.org/produit/3228857000166/100-mie-complet-harrys',
 'https://fr.openfoodfacts.org/produit/3501890002152/soupe-3-poissons-aux-algues-sainte-barbe',
 'https://fr.openfoodfacts.org/produit/7613034626844/nestle-chocapic-cereales-petit-dejeuner',
 'https://fr.openfood

In [32]:
len(recup_url(5))

500

## 2- savoir le temps d'execution de 1000 urls 

In [6]:
start=time.time()
recup_url(1000)
end=time.time()
print('le temps d execution est',(end-start)/(60*60),'hours')

le temps d execution est 0.4755968877342012 hours


In [17]:
liste_url_1=[]

for lien in url[7000:7971]:
    t=requests.get(lien)
    cite=BeautifulSoup(t.text,'html.parser')
    for a in cite('a',href=True):
        if '/produit/' in a['href']:
            liste_url_1.append('https://fr.openfoodfacts.org'+a['href'])

In [18]:
len(liste_url_1)

97100

## code pour récupérer 31 informations pour chaque produits 

In [19]:
info=[]

for url_1 in liste_url_1:
    s=requests.get(url_1)
    cite=BeautifulSoup(s.text,'html.parser')# faire comprendre a python que c'est un text html
    str_1=cite.find('h1').text[0:-2].split()
    nom=' '.join(str_1)
    for i in range(0,len(cite.find_all('span',attrs={'id':'barcode'}))):
        code_bare=cite.find_all('span',attrs={'id':'barcode'})[i].text
        break
    #nutri_score=cite.find_all('tr',attrs={'id':"nutriment_nutriscore_tr", 'class':"nutriment_sub"})[0].contents[3].text.strip() if cite.find_all('tr',attrs={'id':"nutriment_nutriscore_tr", 'class':"nutriment_sub"}) else 'XXX'
    nutr=[]
    for a in cite.find_all('h4'):
        if 'Nutri' in a.text:
            for ele in a.parent.find_all('strong'):
                if 'Nutri-Score' in ele.text:
                    n=ele.text.split()
                    nutr.append(''.join(n))
    if len(nutr)==0:
        nutr='XXX'
    
    nutri_score=''.join(nutr) 
   

    #eco_score=cite.find_all('div',attrs={'id':'ecoscore_drop'})[0].contents[13].contents[2].text.split()[3] if cite.find_all('div',attrs={'id':'ecoscore_drop'})else 'XXX'
    eco=[]
    for a in cite.find_all('h4'):
        if 'Eco-score' in a.text:
            for ele in a.parent.find_all('strong'):
                if 'Note Éco-score' in ele.text:
                    n=ele.text.split()
                    eco.append(''.join(n))
    if len(eco)==0:
        eco_score='XXX'

    
    eco_score=''.join(eco)
    if eco_score=='' :
        eco_score='XXX'
    
             

        
        
    quant=[]
    for ele in cite.find_all('span',attrs={'class':'field'}):
        if 'Quantité'in ele.text:
            quant.append(ele.parent.text.replace(' ','')[10:])
    if len(quant)==0:
        quant='XXX'
    
    
    
    quantité=''.join(quant)
    
    nov=[]
    for ele in cite.find_all('h4'):
        if 'Groupe NOVA' in ele.text:
            if len(ele.parent.find_all('img',alt=True))==1:
                nov.append(ele.parent.find_all('img',alt=True)[0]['alt'])
            
            if len(ele.parent.find_all('img',alt=True))>1: 
                nov.append(ele.parent.find_all('img',alt=True)[1]['alt'])
            
    if len(nov)==0:
        nova='XXX' 
        
    nova=''.join(nov)
    if nova=='':
        nova='XXX'


             
    
    

    
    cond=[]
    for a in cite.find_all('a',href=True):
        if '/conditionnement/'in a['href']:
            cond.append(a.text)
    if len(cond)==0:
        cond= 'XXX'
        
    conditionnement=','.join(cond)# pour ne pas avoir une liste dans la sortie
    
    
    
    if len(cite.find_all('a',attrs={'itemprop':"brand"}))!=0:# un produit peut ne pas avoir la marque mais la balise existe est elle est vide 
        for ele in cite.find_all('a',attrs={'itemprop':"brand"}):
            marque=ele.text
    else:
        marque='XXX'
      
    
    
    cat=[]
    for a in cite.find_all('a',attrs={'class':"tag well_known"}):
        if '/categorie/'in a['href']:
            cat.append(a.text)
    if len(cat)==0 :
        cat='XXX'
    
    categorie=','.join(cat)
        
        
    pays_v=[]
    for a in cite.find_all('a',attrs={'class':"tag well_known"}):
        if '/pays/'in a['href']:
            pays_v.append(a.text)
    if len(pays_v)==0:
        pays_v='XXX'
    
    pays_vente=','.join(pays_v)
        
        
        
    lab=[]    
    for a in cite.find_all('a',attrs={'class':"tag well_known"}):
        if '/label/'in a['href']:
            lab.append(a.text)
    if len(lab)==0:
        lab='XXX'
    label=','.join(lab)    
        
    ori=[]  
    for a in cite.find_all('a',attrs={'class':"tag well_known"}):
        if '/origine/'in a['href']:
            ori.append(a.text)
    if len(ori)==0:
        ori='XXX'
    origine=','.join(ori)    
     
    
    trac=[]
    for a in cite.find_all('a',attrs={'class':"tag well_known"}):
        if '/trace/'in a['href']:
            trac.append(a.text)
    if len(trac)==0:
        trac='XXX'
    trace=','.join(trac)    
    
    magas=[]
    for a in cite.find_all('a',href=True):
        if '/magasin/'in a['href']:
            magas.append(a.text)
    if len(magas)==0:
        magas='XXX'
    magasin=','.join(magas)    
    
   
    add=[]
    for a in cite.find_all('a',href=True):
        if '/additif/'in a['href']:
            add.append(a.text)
            
    if len(add)==0:
        add='XXX'
    additif=','.join(add)    
        
    lieu=[]
    for a in cite.find_all('a',href=True):
        if '/lieu-de-fabrication/'in a['href']:
            lieu.append(a.text)
    if len(lieu)==0:
        lieu='XXX'
    lieu_de_fab=','.join(lieu)   
        
    if len(cite.find_all('div',id=True, property=True))!=0:
        liste_ingredient=cite.find_all('div',id=True, property=True)[0].text if cite.find_all('div',id=True, property=True)[0] else 'XXX'
    else:
        liste_ingredien='XXX'
    analyse_v=[]    
    for i in range(0,len(cite.find_all('span',attrs={'class':'alert round label ingredients_analysis green'}))):
        analyse=cite.find_all('span',attrs={'class':'alert round label ingredients_analysis green'})[i].contents[2].strip() if cite.find_all('span',attrs={'class':'alert round label ingredients_analysis green'}) else 'XXX'
        analyse_v.append(analyse)
    if len(analyse_v)==0:
        analyse_v='XXX'
    
    analyse_vert=','.join(analyse_v)
        
    analyse_R=[]
    for i in range(0,len(cite.find_all('span',attrs={'class':'alert round label ingredients_analysis red'}))):
        
        analyse_r=cite.find_all('span',attrs={'class':'alert round label ingredients_analysis red'})[i].contents[2].strip() if cite.find_all('span',attrs={'class':'alert round label ingredients_analysis red'}) else 'XXX'
        analyse_R.append(analyse_r)
    
    if len(analyse_R)==0:
        analyse_R='XXX'
        
        
    analyse_rouge=','.join(analyse_R)    
    
    analyse_or=[]
    for i in range(0,len(cite.find_all('span',attrs={'class':'alert round label ingredients_analysis orange'}))):
        
        analyse_o=cite.find_all('span',attrs={'class':'alert round label ingredients_analysis orange'})[i].contents[2].strip() if cite.find_all('span',attrs={'class':'alert round label ingredients_analysis orange'}) else 'XXX'
        analyse_or.append(analyse_o)
    
    if len(analyse_or)==0:
        analyse_or='XXX'
        
    analyse_orange=','.join(analyse_or)    
            
    mine=[]
    for a in cite.find_all('a',href=True):
        if '/mineral' in a['href']:
            mine.append(a.text)
    if len(mine)==0:
        mine='XXX'
        
    mineraux=','.join(mine)    
    
    for a in cite.find_all('a',href=True):
        if 'http'in a['href']:
            if 'http'in a.text:
                lien=a.text   
    if len(lien)==0:
        lien='XXX'
    

    
    
    if len(cite.find_all('tr',attrs={ 'id':"nutriment_energy_tr", 'class':"nutriment_main"}))!=0:
        if len(cite.find_all('td',attrs={'class':"nutriment_value"}))!=0:
            for a in cite.find_all('tr',attrs={ 'id':"nutriment_energy_tr", 'class':"nutriment_main"}):
                nb_calorie=a.find('td',attrs={'class':"nutriment_value"}).text.replace(' ','').split()
    else:
        nb_calorie='XXX'
        
        
  
    
    
    
    
    
    
  
    
    
    if len(cite.find_all('label',style=True))>1:
        for i in range (1,len(cite.find_all('label',style=True))):
            comp=cite.find_all('label',style=True)[i].text.strip() if cite.find_all('label',style=True)[i] else 'XXX'
            break
   
    else:
        comp='XXX'
     
    
     
    glut=[]   
    for a in cite.find_all('a',attrs={'class':"tag well_known"}):
        if '/allergene/'in a['href']:
            if 'Gluten'in a.text:
                glut.append(a.text)
    if len(glut)==0:
        glut='XXX'
    gluten=','.join(glut)# pour ne pas avoir en retour une liste dans un tableau
    

    data=[]
    for ele in cite.find_all('h4'):
        if 'Repères nutritionnels pour 100 g' in ele.text:
            for ele in ele.parent.find_all('b'):
                data_1=ele.previous_element.replace(' ','')#la matiere calculé
                data_2=ele.next_element#premier element qui est la quantité
                data_3=ele.next_element.next_element#troisieme element qui est la description
         
                data.append((data_2,data_1,data_3))#stocker dans une liste 
   
    # on va faire un test car dans des pages car on peut trouver une matiere ou 2 matieres ou 3 ou y'a pas d analyse d'ingredient
    
    if len(data)==0:  # y'a pas d'analyse d'ingredient dans la page 
        grasse= 'XXX'
        acide_saturé='XXX'
        sucre='XXX'
        sel='XXX'         


        
    if len(data)==1:# y'a qu'une seul analyse mais ca peut etre une entre 4(grasse-acide-saturé-sucre-sel)
        if 'Matières grasses / Lipides'==data_2[0][0]:
            grasse= data[0][1]+data[0][2]
        else:
            grasse='XXX'
        if 'Acides gras saturés'==data[0][1]:
            acide_saturé=data[0][1]+data[0][2]
        else:
            acide_saturé='XXX'    
        if 'Sucres'==data[0][0]:
            sucre=data[0][1]+data[0][2]
        else:
            sucre='XXX'
        
        if 'Sel'==data[0][0]:
            sel=data[0][1]+data[0][2]
        
    if len(data)==2:#y'a deux analyse mais ca peut etre une entre les 4
        if 'Acides gras saturés'==data[1][0]:
            acide_saturé=data[1][1]+data[1][2]
        else:
            acide_saturé='XXX'  
        
        
        if 'Sucres'==data[1][0]:
            sucre=data[1][1]+data[1][2]
        else:
            sucre='XXX'
        
        if 'Sel'==data[1][0]:
            sel=data[1][1]+data[1][2]
        

    if len(data)==3:#y'a trois analyse mais ca peut etre une entre 4
        if 'Sucres'==data[2][0]:
            sucre=data[2][1]+data[2][2]
        else:
            sucre='XXX'
        if 'Sel'==data[2][0]:
            sel=data[2][1]+data[2][2]

    if len(data)==4:# si il y'a l analyse d'ingredient sur 100 complete 
        grasse= data[0][1]+data[0][2]
        acide_saturé=data[1][1]+data[1][2]
        sucre=data[2][1]+data[2][2]
        sel=data[3][1]+data[3][2]         

        
    if len(cite.find_all('tr',attrs={'id':"nutriment_carbohydrates_tr",'class':"nutriment_main"}))!=0:
        for ele in cite.find_all('td',attrs={'class':"nutriment_label"}):
            if 'Glucides' in ele.text:
                if len(cite.find_all('td',attrs={'class':"nutriment_value"}))!=0:
                    glucides=ele.parent.find_all('td',attrs={'class':"nutriment_value"})[0].text.replace(' ','').replace(',','.').split()[0]
        if glucides==' ':
            if len(cite.find_all('td',attrs={'class':"nutriment_value"}))>1:
                glucides=ele.parent.find_all('td',attrs={'class':"nutriment_value"})[1].text.replace(' ','').replace(',','.').split()[0]
    else:
        glucides='XXX'
        
    if glucides=='?':
        glucides='XXX'    
    
    #if glucide[:-1].isnumeric():
        #glucides=float(glucide[:-1])
    
    
    if len(cite.find_all('tr',attrs={"id":"nutriment_proteins_tr" ,'class':"nutriment_main"}))!=0: #test d existence du composant glucides tableau 
        for ele in cite.find_all('td',attrs={'class':"nutriment_label"}):
                if 'Protéines' in ele.text: 
                    if len(cite.find_all('td',attrs={'class':"nutriment_value"}))!=0:
                        proteines=ele.parent.find_all('td',attrs={'class':"nutriment_value"})[0].text.replace(' ','').replace(',','.').split()[0]
        if proteines==' ':# des fois les informotions nutritionnel passent a la deuxieme colonne
            if len(cite.find_all('td',attrs={'class':"nutriment_value"}))>1:
                proteines=ele.parent.find_all('td',attrs={'class':"nutriment_value"})[1].text.replace(' ','').replace(',','.').split()[0]    
    else:
        proteines='XXX' 
    
    if proteines=='?':
        proteines='XXX'
        
    #if proteine[:-1].isnumeric():
        #proteines=float(proteine[:-1])    
    
    
        
        
    

    if len(cite.find_all('tr',attrs={'id':"nutriment_energy-kcal_tr",'class':"nutriment_main"}))!=0: 
        for ele in cite.find_all('td',attrs={'class':"nutriment_label"}):
            if 'Énergie (kcal)' in ele.text:
                if len(cite.find_all('td',attrs={'class':'nutriment_value'}))!=0:
                    energie_kcal=ele.parent.find_all('td',attrs={'class':'nutriment_value'})[0].text.replace(' ','').split()[0]
        if energie_kcal=='':   
            if len(cite.find_all('td',attrs={'class':'nutriment_value'}))>1:
                energie_kcal=ele.parent.find_all('td',attrs={'class':'nutriment_value'})[1].text.replace(' ','').split()[0]
            
    else :
        energie_kcal='XXX'
    
    if energie_kcal=='?' :
        energie_kcal='XXX'
    
    
    
    #y=filter(str.isdigit,energie)
    #energie_kcal=''.join(y)
        
        
        
    impac=[]
    if len(cite.find_all('table',attrs={'id':"agribalyse_impacts"}))!=0:
        for ele in cite.find_all('table',attrs={'id':"agribalyse_impacts"}):
            for a in ele.find_all('td'):
                r=a.text.split()
                impac.append(','.join(r))
                
        for i in range(1,len(impac)-1):
            if '' in impac :
                impac.remove('')# pour supprimer la 3éme colonne qui est vide 
    
    else:
        impac='XXX'   
        
        
    impact=','.join(impac)    
    
    
   
     
        

    info.append((nom,code_bare,nutri_score,nova,eco_score,quantité,conditionnement,marque,categorie,pays_vente,label,origine,trace,magasin,additif,lieu_de_fab,liste_ingredient,analyse_vert,analyse_rouge,analyse_orange,mineraux,lien,grasse,proteines,glucides,energie_kcal,nb_calorie,acide_saturé,sel
                 ,comp,sucre,gluten,impact))
    

## definition d'une fonction du code Précédent 

In [48]:
def recup_info(liste_url_1):
    info=[]

    for url_1 in liste_url_1:
        s=requests.get(url_1)
        cite=BeautifulSoup(s.text,'html.parser')# faire comprendre a python que c'est un text html
        str_1=cite.find('h1').text[0:-2].split()
        nom=' '.join(str_1)
        for i in range(0,len(cite.find_all('span',attrs={'id':'barcode'}))):
            code_bare=cite.find_all('span',attrs={'id':'barcode'})[i].text
            break
        nutri_score=cite.find_all('tr',attrs={'id':"nutriment_nutriscore_tr", 'class':"nutriment_sub"})[0].contents[3].text.strip() if cite.find_all('tr',attrs={'id':"nutriment_nutriscore_tr", 'class':"nutriment_sub"}) else 'XXX'
        
        
        eco_score=cite.find_all('div',attrs={'id':'ecoscore_drop'})[0].contents[13].contents[2].text.split()[3] if cite.find_all('div',attrs={'id':'ecoscore_drop'})else 'XXX'
    
    
        
        
        quant=[]
        for ele in cite.find_all('span',attrs={'class':'field'}):
            if 'Quantité'in ele.text:
                quant.append(ele.parent.text.replace(' ','')[10:])
        if len(quant)==0:
            quant='XXX'
    
    
    
        quantité=''.join(quant)
    
    
        nova=[]
        for ele in cite.find_all('a',href=True):
            if '/nova' in ele['href']:
                for ele in ele.find_all('img'):
                    nova.append(ele['alt'])
    
    
        if len(nova)==0:
            nova='XXX' 


    
    
    
        cond=[]
        for a in cite.find_all('a',href=True):
            if '/conditionnement/'in a['href']:
                cond.append(a.text)
        if len(cond)==0:
            cond= 'XXX'
        
        conditionnement=','.join(cond)# pour ne pas avoir une liste dans la sortie
    
    
    
        if len(cite.find_all('a',attrs={'itemprop':"brand"}))!=0:# un produit peut ne pas avoir la marque mais la balise existe est elle est vide 
            for ele in cite.find_all('a',attrs={'itemprop':"brand"}):
                marque=ele.text
        else:
            marque='XXX'
      
    
   
    
        cat=[]
        for a in cite.find_all('a',attrs={'class':"tag well_known"}):
            if '/categorie/'in a['href']:
                cat.append(a.text)
        if len(cat)==0 :
            cat='XXX'
    
        categorie=','.join(cat)
        
        
        pays_v=[]
        for a in cite.find_all('a',attrs={'class':"tag well_known"}):
            if '/pays/'in a['href']:
                pays_v.append(a.text)
        if len(pays_v)==0:
             pays_v='XXX'
    
        pays_vente=','.join(pays_v)
        
        
        
        lab=[]    
        for a in cite.find_all('a',attrs={'class':"tag well_known"}):
            if '/label/'in a['href']:
                lab.append(a.text)
        if len(lab)==0:
            lab='XXX'
        label=','.join(lab)    
        
        ori=[]  
        for a in cite.find_all('a',attrs={'class':"tag well_known"}):
            if '/origine/'in a['href']:
                ori.append(a.text)
        if len(ori)==0:
            ori='XXX'
        
        origine=','.join(ori)    
     
    
        trac=[]
        for a in cite.find_all('a',attrs={'class':"tag well_known"}):
            if '/trace/'in a['href']:
                trac.append(a.text)
        if len(trac)==0:
            trac='XXX'
        
        trace=','.join(trac)    
    
        magas=[]
        for a in cite.find_all('a',href=True):
            if '/magasin/'in a['href']:
                magas.append(a.text)
        if len(magas)==0:
            magas='XXX'
        
        
        magasin=','.join(magas)    
    
   
        add=[]
        for a in cite.find_all('a',href=True):
            if '/additif/'in a['href']:
                add.append(a.text)
            
        if len(add)==0:
            add='XXX'
        
        additif=','.join(add)    
        
        lieu=[]
        for a in cite.find_all('a',href=True):
            if '/lieu-de-fabrication/'in a['href']:
                lieu.append(a.text)
        if len(lieu)==0:
            lieu='XXX'
        
        
        lieu_de_fab=','.join(lieu)   
        
        if len(cite.find_all('div',id=True, property=True))!=0:
            liste_ingredient=cite.find_all('div',id=True, property=True)[0].text if cite.find_all('div',id=True, property=True)[0] else 'XXX'
        else:
            liste_ingredien='XXX'
    
        analyse_v=[]    
        for i in range(0,len(cite.find_all('span',attrs={'class':'alert round label ingredients_analysis green'}))):
            analyse=cite.find_all('span',attrs={'class':'alert round label ingredients_analysis green'})[i].contents[2].strip() if cite.find_all('span',attrs={'class':'alert round label ingredients_analysis green'}) else 'XXX'
            analyse_v.append(analyse)
        if len(analyse_v)==0:
            analyse_v='XXX'
    
        analyse_vert=','.join(analyse_v)
        
        analyse_R=[]
        for i in range(0,len(cite.find_all('span',attrs={'class':'alert round label ingredients_analysis red'}))):
            analyse_r=cite.find_all('span',attrs={'class':'alert round label ingredients_analysis red'})[i].contents[2].strip() if cite.find_all('span',attrs={'class':'alert round label ingredients_analysis red'}) else 'XXX'
            analyse_R.append(analyse_r)
    
        if len(analyse_R)==0:
            analyse_R='XXX'
        
        
        analyse_rouge=','.join(analyse_R)    
    
        analyse_or=[]
        for i in range(0,len(cite.find_all('span',attrs={'class':'alert round label ingredients_analysis orange'}))):
            analyse_o=cite.find_all('span',attrs={'class':'alert round label ingredients_analysis orange'})[i].contents[2].strip() if cite.find_all('span',attrs={'class':'alert round label ingredients_analysis orange'}) else 'XXX'
            analyse_or.append(analyse_o)
    
        if len(analyse_or)==0:
            analyse_or='XXX'
        
        analyse_orange=','.join(analyse_or)    
            
        mine=[]
        for a in cite.find_all('a',href=True):
            if '/mineral' in a['href']:
                mine.append(a.text)
        if len(mine)==0:
            mine='XXX'
        
        mineraux=','.join(mine)    
    
        for a in cite.find_all('a',href=True):
            if 'http'in a['href']:
                if 'http'in a.text:
                    lien=a.text   
        if len(lien)==0:
            lien='XXX'
    
        
        
        if len(cite.find_all('tr',attrs={ 'id':"nutriment_energy_tr", 'class':"nutriment_main"}))!=0:
            if len(cite.find_all('td',attrs={'class':"nutriment_value"}))!=0:
                for a in cite.find_all('tr',attrs={ 'id':"nutriment_energy_tr", 'class':"nutriment_main"}):
                    nb_cal=a.find('td',attrs={'class':"nutriment_value"}).text.replace(' ','').split()
        else:
            nb_calorie='XXX'
        
        
  
        nb_calorie=''.join(nb_cal)
        
        if len(cite.find_all('label',style=True))>1:
            for i in range (1,len(cite.find_all('label',style=True))):
                comp=cite.find_all('label',style=True)[i].text.strip() if cite.find_all('label',style=True)[i] else 'XXX'
                break
   
        else:
            comp='XXX'
     
    
    
    
    
     
        glut=[]   
        for a in cite.find_all('a',attrs={'class':"tag well_known"}):
            if '/allergene/'in a['href']:
                if 'Gluten'in a.text:
                    glut.append(a.text)
        if len(glut)==0:
            glut='XXX'
        
        gluten=','.join(glut)# pour ne pas avoir en retour une liste dans un tableau
    

        data=[]
        for ele in cite.find_all('h4'):
            if 'Repères nutritionnels pour 100 g' in ele.text:
                for ele in ele.parent.find_all('b'):
                    data_1=ele.previous_element.replace(' ','')#la matiere calculé
                    data_2=ele.next_element#premier element qui est la quantité
                    data_3=ele.next_element.next_element#troisieme element qui est la description
         
                    data.append((data_2,data_1,data_3))#stocker dans une liste 
   
    # on va faire un test car dans des pages car on peut trouver une matiere ou 2 matieres ou 3 ou y'a pas d analyse d'ingredient
    
        if len(data)==0:  # y'a pas d'analyse d'ingredient dans la page 
            grasse= 'XXX'
            acide_saturé='XXX'
            sucre='XXX'
            sel='XXX'         


        
        if len(data)==1:# y'a qu'une seul analyse mais ca peut etre une entre 4(grasse-acide-saturé-sucre-sel)
            if 'Matières grasses / Lipides'==data_2[0][0]:
                grasse= data[0][1]+data[0][2]
            else:
                grasse='XXX'
            if 'Acides gras saturés'==data[0][1]:
                acide_saturé=data[0][1]+data[0][2]
            else:
                acide_saturé='XXX'    
            if 'Sucres'==data[0][0]:
                sucre=data[0][1]+data[0][2]
            else:
                sucre='XXX'
        
            if 'Sel'==data[0][0]:
                sel=data[0][1]+data[0][2]
        
        if len(data)==2:#y'a deux analyse mais ca peut etre une entre les 4
            if 'Acides gras saturés'==data[1][0]:
                acide_saturé=data[1][1]+data[1][2]
            else:
                acide_saturé='XXX'  
        
        
            if 'Sucres'==data[1][0]:
                sucre=data[1][1]+data[1][2]
            else:
                sucre='XXX'
        
            if 'Sel'==data[1][0]:
                sel=data[1][1]+data[1][2]
        

        if len(data)==3:#y'a trois analyse mais ca peut etre une entre 4
            if 'Sucres'==data[2][0]:
                sucre=data[2][1]+data[2][2]
            else:
                sucre='XXX'
            if 'Sel'==data[2][0]:
                sel=data[2][1]+data[2][2]

        if len(data)==4:# si il y'a l analyse d'ingredient sur 100 complete 
            grasse= data[0][1]+data[0][2]
            acide_saturé=data[1][1]+data[1][2]
            sucre=data[2][1]+data[2][2]
            sel=data[3][1]+data[3][2]         

        
        if len(cite.find_all('tr',attrs={'id':"nutriment_carbohydrates_tr",'class':"nutriment_main"}))!=0:
            for ele in cite.find_all('td',attrs={'class':"nutriment_label"}):
                if 'Glucides' in ele.text:
                    if len(cite.find_all('td',attrs={'class':"nutriment_value"}))!=0:
                        glucides=ele.parent.find_all('td',attrs={'class':"nutriment_value"})[0].text.replace(' ','').replace(',','.').split()[0]
            if glucides==' ':
                if len(cite.find_all('td',attrs={'class':"nutriment_value"}))>1:
                    glucides=ele.parent.find_all('td',attrs={'class':"nutriment_value"})[1].text.replace(' ','').replace(',','.').split()[0]
        else:
            glucides='XXX'
    #if glucide[:-1].isnumeric():
        #glucides=float(glucide[:-1])
    
    
        if len(cite.find_all('tr',attrs={"id":"nutriment_proteins_tr" ,'class':"nutriment_main"}))!=0: #test d existence du composant glucides tableau 
            for ele in cite.find_all('td',attrs={'class':"nutriment_label"}):
                if 'Protéines' in ele.text: 
                    if len(cite.find_all('td',attrs={'class':"nutriment_value"}))!=0:
                        proteines=ele.parent.find_all('td',attrs={'class':"nutriment_value"})[0].text.replace(' ','').replace(',','.').split()[0]
            if proteines==' ':# des fois les informotions nutritionnel passent a la deuxieme colonne
                if len(cite.find_all('td',attrs={'class':"nutriment_value"}))>1:
                    proteines=ele.parent.find_all('td',attrs={'class':"nutriment_value"})[1].text.replace(' ','').replace(',','.').split()[0]    
        else:
            proteines='XXX' 
        
    #if proteine[:-1].isnumeric():
        #proteines=float(proteine[:-1])    
    
    
        
        
    

        if len(cite.find_all('tr',attrs={'id':"nutriment_energy-kcal_tr",'class':"nutriment_main"}))!=0: 
            for ele in cite.find_all('td',attrs={'class':"nutriment_label"}):
                if 'Énergie (kcal)' in ele.text:
                    if len(cite.find_all('td',attrs={'class':'nutriment_value'}))!=0:
                        energie_kcal=ele.parent.find_all('td',attrs={'class':'nutriment_value'})[0].text.replace(' ','').split()[0]
            if energie_kcal=='':   
                if len(cite.find_all('td',attrs={'class':'nutriment_value'}))>1:
                    energie_kcal=ele.parent.find_all('td',attrs={'class':'nutriment_value'})[1].text.replace(' ','').split()[0]
            
        else :
            energie_kcal='XXX'
    
    
    
    #y=filter(str.isdigit,energie)
    #energie_kcal=''.join(y)
        
        
        
        impac=[]
        if len(cite.find_all('table',attrs={'id':"agribalyse_impacts"}))!=0:
            for ele in cite.find_all('table',attrs={'id':"agribalyse_impacts"}):
                for a in ele.find_all('td'):
                    r=a.text.split()
                    impac.append(','.join(r))
                
            for i in range(1,len(impac)-1):
                if '' in impac :
                    impac.remove('')# pour supprimer la 3éme colonne qui est vide 
    
            else:
              impac='XXX'   
        
        
        impact=','.join(impac)    
    
    
   
     
        

    info.append((nom,code_bare,nutri_score,nova,eco_score,quantité,conditionnement,marque,categorie,pays_vente,label,origine,trace,magasin,additif,lieu_de_fab,liste_ingredient,analyse_vert,analyse_rouge,analyse_orange,mineraux,lien,grasse,proteines,glucides,energie_kcal,nb_calorie,acide_saturé,sel
                 ,comp,sucre,gluten,impact))
    return(info)



In [9]:
info 


[('Purée pommes pruneaux sans sucres ajoutés - Charles & Alice - 400 g (4 x 100',
  '3297760020035',
  'Nutri-Score:A',
  '1 - Aliments non transformés ou transformés minimalement',
  'NoteÉco-score:B',
  '400g(4x100g)',
  'Etui,Carton,Pot,Plastique',
  'Charles & Alice',
  "Aliments et boissons à base de végétaux,Aliments d'origine végétale,Desserts,Aliments à base de fruits et de légumes,Fruits et produits dérivés,Compotes,Compotes de pomme,Compotes pommes pruneau",
  'France',
  'Peu ou pas de sucre,Point Vert,Sans sucre ajouté,Sans colorants,Sans conservateurs,Sans édulcorants',
  'X,X,X',
  'X,X,X',
  'X,X,X',
  'E300 - Acide ascorbique',
  'X,X,X',
  'purée de pommes 54.9%, purée de pruneaux 45% (pruneaux 22.5%, eau), antioxydant : acide ascorbique.',
  'Sans huile de palme,Végétalien,Végétarien',
  'X,X,X',
  'X,X,X',
  'X,X,X',
  'http://www.charlesetalice.fr/desserts-go...',
  '0.5g en faible quantité ',
  '0.5g',
  '19g',
  '88kcal',
  ['368kj(88kcal)'],
  '0.1g en faible qua

## temps d'execution du code de récupération 31 infos pour 5 produits 

In [50]:
start=time.time()
recup_info(liste_url_1[0:5])
end=time.time()
print('le temps execution est',end-start,'s' )

le temps execution est 2.0007522106170654 s


## stocker les infos dans un tableux dataframe 

In [24]:
df=pd.DataFrame(result,columns=['Name','code_bare','Nutri_score','nova','Eco','quantité','conditionnement','marque','categorie','pays_vente','Label','Origine','Trace','magasin','Additif','lieu_fab','liste_ingredient','vert','Rouge','Orange','Mineraux','lien','Grasse','proteins','Glucides','Energie_Kcal','nb_calorie','acide_saturé','Sel','Comparaison','Sucre','gluten','impact'])

In [25]:
df

,Name,code_bare,Nutri_score,nova,Eco,quantité,conditionnement,marque,categorie,pays_vente,...,proteins,Glucides,Energie_Kcal,nb_calorie,acide_saturé,Sel,Comparaison,Sucre,gluten,impact
0,Purée pommes pruneaux sans sucres ajoutés - Ch...,3297760020035,Nutri-Score:A,1 - Aliments non transformés ou transformés mi...,NoteÉco-score:B,400g(4x100g),"Etui,Carton,Pot,Plastique",Charles & Alice,"Aliments et boissons à base de végétaux,Alimen...",France,...,0.5g,19g,88kcal,[368kj(88kcal)],0.1g en faible quantité,0.01g en faible quantité,Compotes pommes pruneau,14g en quantité élevée,"X,X,X","Agriculture,32.9%,Transformation,28.7%,Conditi..."
1,Compotes Pommes Rhubarbe sans sucres ajoutés -...,3297760020097,Nutri-Score:A,4 - Produits alimentaires et boissons ultra-tr...,NoteÉco-score:B,388g(4x97g),"Carton,Pots,Plastique",Charles & Alice,"Aliments et boissons à base de végétaux,Alimen...",France,...,<0.5g,8.7g,39kcal,[163kj(39kcal)],0.1g en faible quantité,0.01g en faible quantité,Compotes pommes rhubarbe,7.9g en quantité modérée,"X,X,X","Agriculture,32.9%,Transformation,28.7%,Conditi..."
2,Pommes Passion - Charles & Alice - 388g (4x97,3297760020325,Nutri-Score:A,4 - Produits alimentaires et boissons ultra-tr...,NoteÉco-score:B,388g(4x97g),"X,X,X",Charles & Alice,"Aliments et boissons à base de végétaux,Alimen...",France,...,0.6g,11g,49kcal,[205kj(49kcal)],0.1g en faible quantité,0.01g en faible quantité,Compotes sans sucres,11g en quantité modérée,"X,X,X","Agriculture,35.2%,Transformation,26.4%,Conditi..."
3,Compotes pommes bananes vanille - Charles & Al...,3297760020967,XXX,XXX,XXX,388g,X4,Charles & Alice,"X,X,X",France,...,0.7g,15g,69kcal,[289kj(69kcal)],XXX,XXX,XXX,XXX,"X,X,X","X,X,X"
4,Mousse de Pommes Citrons - Charles & Alice - 4...,3297760080022,Nutri-Score:C,4 - Produits alimentaires et boissons ultra-tr...,XXX,4*48g,"Carton,Pots,Plastique",Charles & Alice,"Produits laitiers,Desserts,Desserts lactés,Mou...",France,...,2g,16.7g,146kcal,[609kj(146kcal)],5.5g en quantité élevée,0.02g en faible quantité,Mousses lactées,15.4g en quantité élevée,"X,X,X","X,X,X"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297095,Rillettes de Thon au poivre ve,3760012693061,Nutri-Score:D,XXX,NoteÉco-score:D,XXX,"X,X,X",XXX,"Produits de la mer,Produits à tartiner,Poisson...",France,...,13g,2.6g,168kcal,[703kj(168kcal)],8.6g en quantité élevée,0.82g en quantité modérée,Rillettes de thon,0.9g en faible quantité,"X,X,X","Agriculture,74.4%,Transformation,12.5%,Conditi..."
297096,Pot au f,0201296044017,XXX,XXX,XXX,XXX,"X,X,X",XXX,"X,X,X",France,...,9.3g,5.3g,76kcal,[318kj(76kcal)],XXX,XXX,XXX,XXX,"X,X,X","X,X,X"
297097,Graine de tournesol - Great Val,0628915830125,Nutri-Score:D,XXX,NoteÉco-score:C,XXX,"X,X,X",Great Value,"Aliments et boissons à base de végétaux,Alimen...",France,...,26g,20g,620kcal,"[2, 594kj(620kcal)]",6g en quantité élevée,1.1g en quantité modérée,Graines de tournesol,2g en faible quantité,"X,X,X","Agriculture,78.2%,Transformation,0.0%,Conditio..."
297098,Tartellette pom,0202279017059,XXX,XXX,XXX,XXX,"X,X,X",XXX,"X,X,X",France,...,XXX,XXX,XXX,XXX,XXX,XXX,XXX,XXX,"X,X,X","X,X,X"


## telecharger les informations dans un fichier csv 

In [3]:
df.to_csv("yuka_scrapping_297100.csv",index=False) ### index=false pour ne pas doubler les indexes 

NameError: name 'df' is not defined

# Analyse des données sur un Échantillon de 300000 produits pout tester les erreurs 


### 1-Distribution du Nutriscore de A à E

In [27]:
df2=[]
for i  in range(0 , len(df)):
    if df['Nutri_score'][i] != 'XXX':# enlever le 'XXX'
        df2.append(df['Nutri_score'][i])
df3 = pd.DataFrame(df2)# je stock la liste dans un tableau 
print(df3.value_counts())

Nutri-Score:D    28968
Nutri-Score:C    19983
Nutri-Score:E    18955
Nutri-Score:A    11240
Nutri-Score:B    11102
dtype: int64


### 2-Distribution du Nova de 1 à 4

In [29]:
df2=[]
for i  in range(0 , len(df)):
    if df['nova'][i] != 'XXX': # enlever le 'XXX'
        df2.append(df['nova'][i])
df3 = pd.DataFrame(df2)# je stock la liste dans un tableau 
print(df3.value_counts())

4 - Produits alimentaires et boissons ultra-transformés     35613
3 - Aliments transformés                                    12971
1 - Aliments non transformés ou transformés minimalement     6061
Risque élevé de sur-exposition                               3993
2 - Ingrédients culinaires transformés                       1282
Risque modéré de sur-exposition                               321
dtype: int64


### 3-Distribution du Eco-score de A à E

In [28]:
df2=[]
for i  in range(0 , len(df)):
    if df['Eco'][i] != 'XXX':# enlever le 'XXX'
        df2.append(df['Eco'][i])
df3 = pd.DataFrame(df2)# je stock la liste dans un tableau
print(df3.value_counts())

NoteÉco-score:B    27716
NoteÉco-score:D    27265
NoteÉco-score:C    17858
NoteÉco-score:E    13957
NoteÉco-score:A     3566
dtype: int64


### 4-Liste des elements sans gluten optimaux a nutri_score=A et nova=1 et eco-score=A

In [81]:
for i in range (0,len(df)):
    if (df['gluten'][i]=='X,X,X') & (df['Nutri_score'][i]=='A')& (df['Eco'][i]=='XXX'): # j'ai pris ici eco='XXX'
        for ele in df['nova'][i]:
            x = re.findall("\d+\.\d+|\d+", ele)# chercher le  decimal dans ele 
            if len(x)!=0: # si elle retourne aucun element 
                if (int(x[0])==1) :
                    print(df['Name'][i])

Cristaline Eau de source - 1,5
naturelle volcanique - volvic - 1,5
Hépar - 1 lit
san Pellegrino - 1
Eau minérale - Vittel - 1,5
Evian 1.
Badoit® - 1
Eau - Contrex - 1,5
Mélange de fruits secs - Alesto - 20
La Salvetat - 1.25
Eau minérale naturelle - Mont Roucous -
Rozana - 1
Eau de source - Saguaro - 2
Eau de source gazéifiée Cristaline - 1,5
Eau de source de montagne Laqueuille - Marque repère - 1.5
Evian - 1lit
Courmayeur - Eau minérale naturelle - 1,5
Purée amande complète - Jean Hervé - 350
Perrier fines bulles - 1
Eau de source - Cristaline - 1
Vichy - Vichy Célestins - 1,25
Eau d'Auvergne - Carrefour - 1.5
Eau minérale Contrex - 1


### 5-Nombre de produit ayant le nombre de calorie superieur a 500

In [30]:
import re
for i in range(0,len(df)):
    x = re.findall("\d+\.\d+kj|\d+kj", df['nb_calorie'][i])
    if len(x) != 0 : # test pour la valeur nulles de l'energie 
        y = float(x[0][:-2])# slicing pour enlever KJ
        if(y>500):
            print(df['Name'][i])

Nutella - Ferrero - 400
Prince Chocolat - Lu - 300
Nutella - Ferrero - 1
Muesli Raisin, Figue, Abricot - Bjorg - 375 g
Biscuit Sésame - Gerblé - 230
Nutella biscuits - Ferrero - 304
100% mie complet - Harrys - 500
NESTLE CHOCAPIC Céréales Petit Déjeuner - 430
Nocciolata Pâte à tartiner au cacao et noisettes - Rigoni di Asiago - 270
Pain 100% mie nature PT - Harrys - 500
Nesquik - Nestlé - 1
Flocons d'avoine complète - Bjorg - 50
Excellence 70% Cacao Noir Intense - Lindt - 100 g
Fourrés Chocolat noir - bjorg - 225 g (3 sachet
Special Muesli 30% fruits & noix céréales complètes - Jordans - 750
Pains au lait - Pasquier - 350
Biscuit lait chocolat - Gerblé - 230
Quaker Cruesli Mélange de noix - 450
Pain de mie à la farine complète - La Boulangère Bio - 500
Country Crisp 4 Noix - Jordans - 550
Biscuit pomme noisette - Gerblé - 230
Biscottes 6 Céréales - Heudebert - 300 g
Primevère Bio doux Tartine & Cuisson - 250
Pur beurre de cacahuète - Jardin Bio - 350
American Sandwich Complet - Harrys 

### 6- Nombre de produit ayant le ratio sucre supérieure à 0,6

In [23]:
count = 0
for i in range(0,len(df)):
    x = re.findall("\d+\.\d+|\d+", df['Sucre'][i]) #retoune un tableau de taille 1 de la valeur du sucre
    if len(x) != 0 : # test pour la valeur nulles du sucre 
        ratio = float(x[0])/100 # calcul ratio du sucre par 100g
    if ratio > 0.6 :
        count = count+1 # somme des elements superieurs a 0.6
print('- Le nombre de produit avec un ratio superieur a 0.6 est : ',count)

- Le nombre de produit avec un ratio superieur a 0.6 est :  24


### 7- Liste de produits sans huile de palme et vendu en France et en belgique 

In [24]:
print('les produit ......sont :\n\n')
for i in range (0,len(df)):
    if('France' in df['pays_vente'][i])& ('Belgique'in df['pays_vente'][i])&('Huile de palme' in df['Rouge'][i]):
        print(df['Name'][i])

les produit ......sont :


Nutella - Ferrero - 400
Prince Chocolat - Lu - 300
Nutella - Ferrero - 1
Nutella biscuits - Ferrero - 304
Biscottes 6 Céréales - Heudebert - 300 g
Nutella - Ferrero - 975
Nutella - Ferrero - 750
Belvita Petit Déjeuner Original Chocolat - LU - 400 g (8× 50 g
Nutella - Ferrero - 825
Cracotte Céréales Complètes - lu - 2
Céréales Extra Pepites Kellogg's Chocolat Noisettes - 500
Cracotte - LU - 250 g (2x 1
Napolitain l'Original - Lu - 6 x 30
bn chocolat -
Kinder bueno gaufrettes enrobees de chocolat 2 barres - 43
Biscotte Fibres+ - Heudebert - 280g, 2 sachets de 16 biscott
Céréales Extra Kellogg's Pépites Chocolat au lait - 500
Spéculoos à tartiner - Lotus - 40
Ferrero rocher fines gaufrettes enrobees de chocolat au lait et noisettes avec noisette entiere boite de 30 pieces - 375
Oreo Original - MONDELEZ - 154
Monaco - Belin - 100 g
Cracotte Chocolat - LU - 200
Pim's L'Original Orange - Lu - 150 g
Kinder maxi barre chocolat au lait avec fourrage au lait 11 barres

### 7-La distribution par nombre de pays 

In [25]:
for i in range(0,len(df)):
    print('le produit',df['Name'][i],'est distribué dans',len(df['pays_vente'][i]),'pays\n')

le produit Cristaline Eau de source - 1,5 est distribué dans 151 pays

le produit Nutella - Ferrero - 400 est distribué dans 153 pays

le produit Prince Chocolat - Lu - 300 est distribué dans 165 pays

le produit Coca-Cola - 330 est distribué dans 314 pays

le produit Nutella - Ferrero - 1 est distribué dans 97 pays

le produit Muesli Raisin, Figue, Abricot - Bjorg - 375 g est distribué dans 129 pays

le produit Biscuit Sésame - Gerblé - 230 est distribué dans 61 pays

le produit Nutella biscuits - Ferrero - 304 est distribué dans 100 pays

le produit 100% mie complet - Harrys - 500 est distribué dans 55 pays

le produit Soupe 3 poissons aux algues - Sainte Bar est distribué dans 6 pays

le produit NESTLE CHOCAPIC Céréales Petit Déjeuner - 430 est distribué dans 108 pays

le produit Nocciolata Pâte à tartiner au cacao et noisettes - Rigoni di Asiago - 270 est distribué dans 82 pays

le produit Pain 100% mie nature PT - Harrys - 500 est distribué dans 55 pays

le produit Nesquik - Nestl

le produit Innocent FA-BU-BLEU - 750 est distribué dans 13 pays

le produit Muesli Crunchy Avoine Chocolat - Alnatura - 40 est distribué dans 65 pays

le produit Spécialité végétale fermentée au lait de coco nature - Andros - 400 est distribué dans 13 pays

le produit KINDER CARDS CHOCOLAT BISCUIT CROUSTILLANT AVEC FOURRAGE ONCTUEUX AU LAIT ET AU CACAO (2x5) - 128 est distribué dans 73 pays

le produit Panzani - spf - sauce pesto vert - 200 est distribué dans 22 pays

le produit le cookie Chocolat noisettes - Bjorg - 200 est distribué dans 17 pays

le produit Seigle & Graines - la boulangère - 500 g (14 tranche est distribué dans 6 pays

le produit Mikado chocolat au lait - LU - 90 est distribué dans 30 pays

le produit Albacore - Thon entier au naturel - petit navire - 93 est distribué dans 23 pays

le produit La Vache qui rit 16 portions - Bel - 280 est distribué dans 15 pays

le produit 4 baguettes viennoises - La Boulangère - 340 est distribué dans 6 pays

le produit Barilla pates 

le produit Kinder schokobons bonbons de chocolat au lait fourres lait et noisettes sachet - 225 g (225 est distribué dans 6 pays

le produit Pâte feuilletée - Herta - 230 est distribué dans 6 pays

le produit Mini BN - 25 biscuits (5 paquet est distribué dans 33 pays

le produit Pâte à tartiner aux noisettes - Alter Eco - 270 est distribué dans 14 pays

le produit Lentilles cuisinées à l'Auvergnate - Raynal et Roquelaure - 820 est distribué dans 17 pays

le produit PurSoup' Velouté de 10 légumes - liebig - 1 est distribué dans 22 pays

le produit Quaker Cruesli Chocolat noir maxi format - 900 est distribué dans 20 pays

le produit Fromage blanc au lait de brebis - Vrai - 40 est distribué dans 6 pays

le produit Grany Chocolat 5 Céréales - Lu - 125 est distribué dans 17 pays

le produit Lait demi-écremé brique - 6 x 1 l UNITÉ - Délisse - 1000 est distribué dans 24 pays

le produit Magnum Amande - Miko - 328 est distribué dans 6 pays

le produit Galettes au bon beurre bio - SAINT MICHEL 

### 8- liste des produits glucides :

In [38]:
for i in range(0,len(df)):
    x = re.findall("\d+\.\d+|\d+", df['Glucides'][i])#retoune un tableau de taille 1 de la valeur du sucre
    y = re.findall("\d+\.\d+|\d+", df['proteins'][i])
    #z = re.findall("\d+\.\d+|\d+", df['Grasse'][i])
    if (len(x) != 0) & (len(y) != 0) & (len(z) != 0) :
        if (float(x[0])>10*float(y[0])): #& ((float(x[0])>10*float(z[0]))):
            print(df['Name'][i])

Prince Chocolat - Lu - 300
Coca-Cola - 330
Nesquik - Nestlé - 1
Céréales Corn Flakes Kellogg's Original - 500
Tropicana Pure premium orange sans pulpe 1 L - 1000
Patamilka aux noisettes - Milka - 600
lipton - 1,5
Galettes St Michel - 130 g (20 unité
Tomato Ketchup BIO - Heinz - 580
Napolitain l'Original - Lu - 6 x 30
SIROP D'agave - Sunny via - 350
Oasis tropical -
bn chocolat -
COOKIE CRISP Céréales - Nestlé - 375
Miel Pops - Kellogg's - 400
Granola Gros éclats de Chocolat - LU - 276
Tropicana Pure premium orange sans pulpe format familial 1,5 L - 1500
Coca Cola gout original - 1,25
Original taste - Coca-Cola - 1.75 L
Pâte à tartiner - Poulain - 400
Noir Extra - Poulain - 100
Coca-Cola - 1
Benco original -
Grand Arôme 32% de Cacao - Poulain - 800
Spéculoos à tartiner - Lotus - 40
Tomato Ketchup - Heinz - 570 g - 500
Pom'Potes (Pomme) - Materne - 90 g
Fleur de Maïs - Maïzena - 400
Oreo Original - MONDELEZ - 154
Fraise Intense - Bonne Maman - 335
Pur sirop d'érable - Maple joe - 25
Lait

### 9-Liste des produits proteiné:

In [40]:
for i in range(0,len(df)):
    x = re.findall("\d+\.\d+|\d+", df['Glucides'][i])#retoune un tableau de taille 1 de la valeur du sucre
    y = re.findall("\d+\.\d+|\d+", df['proteins'][i])
    #z = re.findall("\d+\.\d+|\d+", df['Grasse'][i])
    if (len(x) != 0) & (len(y) != 0) & (len(z) != 0) :
        if (float(y[0])>10*float(x[0])): #& ((float(x[0])>10*float(z[0]))):
            print(df['Name'][i])

Le Bon PARIS à l'étouffée SANS NITRITE 4 tranches - Herta - 14
Caprice des dieux - 300
Lait d'amande Sans sucres - Alpro - 1
Le supérieur cuit à l'étouffée -25% sel* - 4 tranches - Fleury Michon - 160
Camembert - Président - 250
Emmental Cœur de Meule - Président - 250
Bûche Sainte-Maure - Soignon - 200
Carrée frais 0% - Elle & Vire - 200g (8x25
Blanc de Poulet - 25% de Sel* - Fleury Michon - 160
Coulommiers Doux et Crémeux - Cœur de Lion - 350 g
Le bon Paris -25% de sel - Herta - 140 g (4 tr
Sardines à l'huile d'olive - Parmentier - 135 g (net égoutté 95
La Bûche fondante - Président - 180
ZERO NITRITE - LE SUPERIEUR - Fleury Michon - 120
chaussée aux moines - 340
Caprice des dieux - 200
Blanc de Poulet - Doré au Four - Fleury Michon - 160
Emmental Râpé Fondant - Président - 200
Le bon Paris - Herta - 140 g (4 x 35
Albacore - Thon entier au naturel - petit navire - 93
Leerdammer L'Original 8 tranches - LEERDAMER - 200
Yaourt soja nature - Alpro - 500
Carré frais ail et fines herbes - 

9-  Liste des produits de lipides:

In [41]:
for i in range(0,len(df)):
    x = re.findall("\d+\.\d+|\d+", df['Glucides'][i])#retoune un tableau de taille 1 de la valeur du sucre
    y = re.findall("\d+\.\d+|\d+", df['proteins'][i])
    z = re.findall("\d+\.\d+|\d+", df['Grasse'][i])
    if (len(x) != 0) & (len(y) != 0) & (len(z) != 0) :
        if (float(x[0])>10*float(y[0])) & ((float(x[0])>10*float(z[0]))):
            print(df['Name'][i])

Coca-Cola - 330
Nesquik - Nestlé - 1
Céréales Corn Flakes Kellogg's Original - 500
Tropicana Pure premium orange sans pulpe 1 L - 1000
lipton - 1,5
Tomato Ketchup BIO - Heinz - 580
SIROP D'agave - Sunny via - 350
Oasis tropical -
COOKIE CRISP Céréales - Nestlé - 375
Miel Pops - Kellogg's - 400
Tropicana Pure premium orange sans pulpe format familial 1,5 L - 1500
Coca Cola gout original - 1,25
Coca-Cola - 1
Grand Arôme 32% de Cacao - Poulain - 800
Tomato Ketchup - Heinz - 570 g - 500
Pom'Potes (Pomme) - Materne - 90 g
Fleur de Maïs - Maïzena - 400
Fraise Intense - Bonne Maman - 335
Pur sirop d'érable - Maple joe - 25
Céréales Coco Pop's Original - KELLOG'S - 550
Tomato Ketchup - Heinz - 220
Céréales Lion caramel & chocolat - Nestlé - 480
Ice Tea pêche - Lipton ice tea -
Céréales Smacks - Kellogg's - 400
FROSTIES - Kellogg's - 400
Boisson Avoine - Bjorg - 1
Ketchup - Amora - 550
Casse-croûte original - BN - 375
Thé glacé pêche intense - Fuze Tea - 1.2
Tartimiel - Lune de Miel - 250
Stévi

### 10-   Liste des produit ayant 0 kcal considere comme l eau qu on peut le consommer sans augmenter Insuline  

In [47]:
for i in range(0,len(df)):
    x = re.findall("\d+\.\d+|\d+", df['nb_calorie'][i])#retoune un tableau de taille 1 de la valeur du sucre
    if (len(x) != 0)  :
        if float(x[0])== 0:
            print(df['Name'][i])


Cristaline Eau de source - 1,5
Hépar - 1 lit
Badoit® - 1
Eau - Contrex - 1,5
Eau minérale - La Salvetat - 1,15
Coca Zéro - Coca-Cola - 1 lit
La Salvetat - 1.25
Eau minérale naturelle - Mont Roucous -
Rozana - 1
Coca Cola zéro - 330
Sel fin iodé - Cerebos - 500
Eau de source de montagne Laqueuille - Marque repère - 1.5
Evian - 1lit
Sel fin - Chantesel - 75
La Salière - Sel fin iodé - La baleine - 125
Sel de mer - La baleine - 60
Sel Moulu de Méditerranée - la baleine - 250
Courmayeur - Eau minérale naturelle - 1,5
Coca-cola zéro - 500
Sel Fin - La baleine - 550
Perrier fines bulles - 1
NESCAFE Sélection, Café Soluble, Flacon de - 200
St-Yorre eau minérale naturelle naturellement gazeuse - Saint Yorre - 1.1
Vichy - Vichy Célestins - 1,25
Eau d'Auvergne - Carrefour - 1.5
Eau minérale Contrex - 1
